**Working Dir**: It's supposed that your commands are run under this folder.

In [ ]:
WORKING_DIR = u"/path/to/folder/to/music"

**Filename**: This is the filename prefix. For example, if your files are `CDImage.wav`, `CDImage.cue`, set `FILENAME_PREFIX` to `CDImage`.

**Filename Extension**: It's the extension part of your audio file. If your audio file is `CDImage.wav`, set `FILENAME_EXTENSION` to `wav`.

In [ ]:
FILENAME_PREFIX = u"filename_without_ext"
FILENAME_EXTENSION = u"wav"

** Output Prefix**: The output files will be saved to here.

In [ ]:
OUTPUT_PATTERN = u"/path/to/your/music/<%(prefix)s >%(album)s< (%(suffix)s)>/<<%(discnumber)s->%(tracknumber)s >%(title)s.flac"

**Others**: If you have cover picture, set `PICTURE` to that filename.
If your cue is not utf-8 encoded, set `ANSI_ENCODING` to the encoding of your cue sheet file.
At the end of conversion, all files defined in `FILES_TO_COPY` will be simply copied to the output position.

In [ ]:
PICTURE = u"Folder.jpg"
ANSI_ENCODING = "gbk"
FILES_TO_COPY = ["Artworks.tar"]

DELETE_TARGET_DIR = False  # If clean the target folder at first

INPUT_EXTRAINFO = u"%s.ini" % FILENAME_PREFIX
INPUT_CUE = u"%s.cue" % FILENAME_PREFIX
INPUT_AUDIO = u"%s.%s" % (FILENAME_PREFIX, FILENAME_EXTENSION)

In [ ]:
import sys
sys.path.append(u"/path/to/your/GatesMusicPet/")

In [ ]:
from music_pet.meta import *
from music_pet.utils import *
from music_pet.audio import FLAC, init_flacs

import subprocess
import os, sys

In [ ]:
cd $WORKING_DIR

In [ ]:
global_report = []

NOT_PARSED = 1
NO_TRACK = 2

### Parse CUE

In [ ]:
albumList = parse_cue(INPUT_CUE, encoding="U8")

In [ ]:
extraMetas = parse_ini(INPUT_EXTRAINFO)

In [ ]:
for album in albumList.values():
    for extraMeta in extraMetas:
        album.update_all_tracks(extraMeta)

In [ ]:
albumList.fix_album_names()

In [ ]:
flacs = []

In [ ]:
for album in albumList.values():
    flacs = init_flacs(album, OUTPUT_PATTERN)
    for flac in flacs:
        flac.set_input_file(u"%s/%s" % (
                WORKING_DIR, filename_safe(flac.get_tag(u"original_file"))))
        flac.set_next_start_time_from_album(album)
        flac.cover_picture = PICTURE
    for l in album.detail():
        print(l)

In [ ]:
commands = []
tmpified_files = {}

for flac in flacs:
    b_is_wav = flac.get_tag(u"@input_fullpath").endswith(u".wav") 
    b_tempified = flac.get_tag(u"@input_fullpath") in tmpified_files
    if not b_is_wav and not b_tempified:
        commands.append(flac.command_build_tempwav(memoize=tmpified_files))
    commands.append(flac.command())
    commands.append(command_copy_to([PICTURE] + FILES_TO_COPY, parent_folder(flac.get_tag(u"@output_fullpath"))))
    if not b_is_wav and not b_tempified:
        commands.append(flac.command_clear_tempwav())
    flac.create_target_dir()
    
for cmd in commands:
    print(cmd)
    print(u"")


### Covert Files

In [ ]:
cd $WORKING_DIR

In [ ]:
for cmd in commands:
    print(u"Executing:\n%s\n\n" % cmd)
    try:
        p = subprocess.check_output(cmd,
                                    shell=True,
                                    )
    except subprocess.CalledProcessError as ex:
        p = u"Process received an error! code=%s, output=%s" % (ex.returncode, ex.output)
        global_report.append((3, u"Process Error, code=%s" % ex.returncode, cmd))
    print(p)
    print(u"\n\n")

In [ ]:
for error in global_report:
    print(u"%s\n%s\n\n" % (error[1], error[2]))

### *playground*